# Load Packages

What we can recall from critisicm regarding this presentation:

- did not take the equal weighted as benchmark
- use total return indices (I think)

In [1]:
#### Import required Packages ####
import numpy as np
import pandas as pd
import seaborn as sb
import datetime as dt
import matplotlib.pyplot as plt
from pandas.tseries.offsets import MonthEnd

# Seaborn
sb.set_style("ticks")
sb.mpl.rc("figure", figsize=(16,8))
sb.mpl.rc("font", size=14)
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'

# little function for later
def isNaN(string):
    return string != string

# Load Data

In [2]:
# rawdata is direct import from excel

# Import of excel data, sheet by sheet
xls_ETF = pd.ExcelFile("1_Data/Data Clean TM/Data_ETFxIndex_stiched.xlsx")
xls_Inflation = pd.ExcelFile("1_Data/Data Clean TM/Data_Inflation.xlsx")
xls_Yield = pd.ExcelFile("1_Data/Data Clean TM/Yields_Clean_TM.xlsx")
xls_FX = pd.ExcelFile("1_Data/Data Clean TM/Data_FX_spot.xlsx")

ETF_raw = pd.DataFrame(pd.read_excel(xls_ETF))
Inflation_raw = pd.DataFrame(pd.read_excel(xls_Inflation, 2))
Yield_long_raw = pd.DataFrame(pd.read_excel(xls_Yield, 2))
Yield_short_raw = pd.DataFrame(pd.read_excel(xls_Yield, 3))
FX_raw = pd.DataFrame(pd.read_excel(xls_FX, 3))

# keep a safe copy of the rawdata to compare the changes
ETF = ETF_raw.copy()
Inflation = Inflation_raw.copy()
Yield_long = Yield_long_raw.copy()
Yield_short = Yield_short_raw.copy()
FX = FX_raw.copy()

# Set date as index
ETF.set_index("Datum", inplace=True)
Inflation.set_index("Datum", inplace=True)
Yield_long.set_index("Datum", inplace=True)
Yield_short.set_index("Datum", inplace=True)
FX.set_index("Dates", inplace=True)

In [ ]:
# Drop all the columns in which only text is and safe them separately --> ONLY RUN ONCE
# ETF_text = ETF.iloc[0,:]
# Yield_text = Yield.iloc[0,:]
# Yield_short_text = Yield_short.iloc[0,:]
# cpicore_text = cpicore.iloc[0:1,:]
# ETF.drop([0], inplace = True)
# Yield.drop([0], inplace = True)
# Yield_short.drop([0], inplace = True)
# cpicore.drop([0,1], inplace = True)

# Value

Recent infl – the average of the headline and core annual CPI inflation rate
--> we don't have headline inflation


Effective target infl - The effective inflation target is the mean of the target range announced or implied by the authorities plus an adjusted for past “target misses”, which is the last 3 years’ average gap between actual inflation and the target means
--> we don't have target rates for all countries --> use 2.5 as target rate

Formula: (1/n)*recent infl + ((n-1)/n)*effective target infl

In [3]:
# first we will take the difference between the effective target (mean) inflation and the actual inflation

cpiheadline = Inflation.copy()

eff_target_diff = pd.DataFrame(columns = cpiheadline.columns, index = cpiheadline.index)

for i in eff_target_diff.columns:
    if i == "Australia":
        eff_target_diff.loc[:,i] = cpiheadline.loc[:,i] - 0.025
    elif i == "Canada":
        eff_target_diff.loc[:,i] = cpiheadline.loc[:,i] - 0.02
    elif i == "China":
        eff_target_diff.loc[:,i] = cpiheadline.loc[:,i] - 0.02
    elif i == "Germany":
        eff_target_diff.loc[:,i] = cpiheadline.loc[:,i] - 0.02
    elif i == "France":
        eff_target_diff.loc[:,i] = cpiheadline.loc[:,i] - 0.02
    elif i == "UK":
        eff_target_diff.loc[:"2003-11-28",i] = cpiheadline.loc[:"2003-11-28",i] - 0.025
        eff_target_diff.loc["2003-12-31":,i] = cpiheadline.loc["2003-12-31":,i] - 0.02
    elif i == "Italy":
        eff_target_diff.loc[:,i] = cpiheadline.loc[:,i] - 0.02
    elif i == "Japan":
        eff_target_diff.loc[:,i] = cpiheadline.loc[:,i] - 0.033
    elif i == "Singapore":
        eff_target_diff.loc[:,i] = cpiheadline.loc[:,i] - 0.02
    elif i == "USA":
        eff_target_diff.loc[:"2011-12-31",i] = cpiheadline.loc[:"2011-12-31",i] - 0.0185
        eff_target_diff.loc["2012-01-31":,i] = cpiheadline.loc["2012-01-31":,i] - 0.02
    elif i == "Spain":
        eff_target_diff.loc[:,i] = cpiheadline.loc[:,i] - 0.02
    elif i == "Switzerland":
        eff_target_diff.loc[:,i] = cpiheadline.loc[:,i] - 0.01
    elif i == "India":
        eff_target_diff.loc[:,i] = cpiheadline.loc[:,i] - 0.04

<ipython-input-3-e12819c2dedb>:35: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  eff_target_diff.loc[:,i] = cpiheadline.loc[:,i] - 0.04
<ipython-input-3-e12819c2dedb>:26: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  eff_target_diff.loc[:,i] = cpiheadline.loc[:,i] - 0.02
<ipython-input-3-e12819c2dedb>:11: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.iseti

In [10]:
np.nanmean(eff_target_diff.loc[:"2002-11-30", "Singapore"])
#np.nanmean(eff_target_diff.loc["2020-04-30":, "Singapore"])

-0.012722222222222225

In [4]:
# create the rolling 3-year average of the core inflation

# first I have to set up an empty dictionary to store the rolling averages
cpiheadline_avg = {}
for i in cpiheadline.columns:
    cpiheadline_avg[i] = []

# next up we iterate over the cpicore data to get the index and safe them in the dictionary
runner = len(cpiheadline.index)
for i in cpiheadline.index:
    if runner == 35:
        break
    
    for j,k in enumerate(cpiheadline.columns):
        inflation = np.nanmean(eff_target_diff.iloc[runner-36:runner, j])
        cpiheadline_avg[k].insert(0, inflation)
    runner -= 1
    
#cpiheadline_avg
cpiheadline_avg_trimmed = pd.DataFrame(cpiheadline_avg)

<ipython-input-4-718e0d68af44>:15: RuntimeWarning: Mean of empty slice
  inflation = np.nanmean(eff_target_diff.iloc[runner-36:runner, j])


In [8]:
cpiheadline_avg_trimmed # until here I checked several times whether it is correct and it seems to work

,India,Singapore,Canada,USA,Japan,Australia,China,Switzerland,UK,Spain,France,Germany,Italy
Datum,,,,,,,,,,,,,
2002-11-30,NaN,-0.012722,0.004139,0.007306,-0.040861,0.013686,-0.018914,0.001194,-0.014194,0.014091,-0.003139,-0.003778,0.005556
2002-12-31,NaN,-0.013000,0.004722,0.007167,-0.040667,0.013500,-0.019139,0.000972,-0.014028,0.014500,-0.002778,-0.003833,0.005778
2003-01-31,NaN,-0.013278,0.005083,0.007083,-0.040500,0.013833,-0.019139,0.000750,-0.013889,0.014923,-0.002500,-0.004000,0.006000
2003-02-28,NaN,-0.013444,0.005694,0.006917,-0.040444,0.014167,-0.018917,0.000528,-0.013694,0.015071,-0.002389,-0.004111,0.006139
2003-03-31,NaN,-0.013694,0.006222,0.006694,-0.040361,0.014500,-0.018806,0.000361,-0.013444,0.015267,-0.002056,-0.004167,0.006194
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-30,0.021503,0.004167,0.014000,0.023944,-0.026639,0.002667,-0.000500,-0.002639,0.016472,0.014083,0.001833,0.013972,0.008917
2022-12-31,0.021597,0.005861,0.015278,0.025333,-0.025722,0.004111,-0.001306,-0.001778,0.019028,0.015861,0.003278,0.016444,0.012083
2023-01-31,0.021144,0.007444,0.016417,0.026500,-0.024833,0.005639,-0.002056,-0.001056,0.021583,0.017222,0.004528,0.018417,0.015167


In [7]:
# now we calculate the value formula:
# (1/n)*recent inflation + ((n-1)/n)*effective target inflation for the years: 2, 5, 10, 20, 30

# first we need to align the dataframes 
cpiheadline_trimmed = cpiheadline[35:]
cpiheadline_avg_trimmed.set_index(cpiheadline_trimmed.index, inplace = True)

# second we create an empty dictionary to fill with the calculations
formula_values = {}
x = 0
for i in cpiheadline_trimmed.columns:
    x += 1
    for j in [2,5,10,20,30]:
        formula_values[f"{i},{j}"] = []
formula_values = pd.DataFrame(columns = formula_values.keys(), index = cpiheadline_trimmed.index)


# now that we have the empty dictionary to fill, we start with caluclating the inflation expectation
for i,k in enumerate (cpiheadline_trimmed.columns):
    for j,m in enumerate([2,5,10,20,30]):
        formula_values.iloc[:,j+i*5] = (1/m)*cpiheadline_trimmed[k] +((m-1)/m)*cpiheadline_avg_trimmed[k]
        
#formula_values

<ipython-input-7-90c02453a451>:21: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  formula_values.iloc[:,j+i*5] = (1/m)*cpiheadline_trimmed[k] +((m-1)/m)*cpiheadline_avg_trimmed[k]


In [11]:
# now we have the issue that the Yield_clean data and the Inflation_clean data doesn't have the same sequence of countries
# to avoid this we resort the Yield_clean data to fit the Inflation_clean data

clean_names = ["Germany", "France","Spain","Italy","United Kingdom", "Switzerland", "China", "Australia", "Japan", "USA", "Canada", "Singapore", "India"]

# we split the dataframe into the different countries...
Germany = Yield_long.iloc[:,:5]
France = Yield_long.iloc[:,5:10]
Spain = Yield_long.iloc[:,10:15]
Italy = Yield_long.iloc[:,15:20]
UK = Yield_long.iloc[:,20:25]
Switzerland = Yield_long.iloc[:,25:30]
China = Yield_long.iloc[:,30:35]
Australia = Yield_long.iloc[:,35:40]
Japan = Yield_long.iloc[:,40:45]
USA = Yield_long.iloc[:,45:50]
Canada = Yield_long.iloc[:,50:55]
Singapore = Yield_long.iloc[:,55:60]
India = Yield_long.iloc[:,60:]

# ... and fusion them together to have the same order as the Inflation data (plus dividing by 100 to have same level as Inflation)
Yields_aligned = [India, Singapore, Canada, USA, Japan, Australia, China, Switzerland, UK, Spain, France, Germany, Italy]
Yields_aligned = pd.concat(Yields_aligned, axis=1)
Yields_aligned = Yields_aligned /100
#Yields_aligned

In [12]:
# in a next step we adjust the yield data to have the same lookback as the other data

Yield_trimmed = Yields_aligned[35:]

Real_yield = pd.DataFrame(0, columns = formula_values.columns, index = formula_values.index)

# now we take the difference between yield etf and inflation
for i,k in enumerate(formula_values.columns):
    for j,l in enumerate(formula_values.index):
        Real_yield.iloc[j,i] = Yield_trimmed.iloc[j,i] - formula_values.iloc[j,i]
#Real_yield

In [13]:
# in a final step we will average over the duration of the countries to get one average for each country

Real_yield_avg = pd.DataFrame(columns = cpiheadline_avg_trimmed.columns, index = Real_yield.index)

runner = 0
for i in range(0, len(Yields_aligned.columns)-1, 5):
    temp = pd.DataFrame(np.nanmean([Real_yield.iloc[:,i],Real_yield.iloc[:,i+1],Real_yield.iloc[:,i+2],Real_yield.iloc[:,i+3],Real_yield.iloc[:,i+4]], axis = 0))
    Real_yield_avg.iloc[:,runner] = temp.values
    runner +=1
    
#Real_yield_avg

<ipython-input-13-32f040c4ca01>:7: RuntimeWarning: Mean of empty slice
  temp = pd.DataFrame(np.nanmean([Real_yield.iloc[:,i],Real_yield.iloc[:,i+1],Real_yield.iloc[:,i+2],Real_yield.iloc[:,i+3],Real_yield.iloc[:,i+4]], axis = 0))
<ipython-input-13-32f040c4ca01>:8: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  Real_yield_avg.iloc[:,runner] = temp.values
<ipython-input-13-32f040c4ca01>:7: RuntimeWarning: Mean of empty slice
  temp = pd.DataFrame(np.nanmean([Real_yield.iloc[:,i],Real_yield.iloc[:,i+1],Real_yield.iloc[:,i+2],Real_yield.iloc[:,i+3],Real_yield.iloc[:,i+4]], axis = 0))


,India,Singapore,Canada,USA,Japan,Australia,China,Switzerland,UK,Spain,France,Germany,Italy
Datum,,,,,,,,,,,,,
2002-11-30,NaN,0.038142,0.040897,0.049712,0.045416,0.040049,NaN,0.023939,0.054983,0.026126,0.043910,0.045212,0.037274
2002-12-31,NaN,0.034024,0.035837,0.044641,0.043797,0.035000,NaN,0.021170,0.051624,0.022888,0.040268,0.042709,0.033574
2003-01-31,NaN,0.033443,0.038607,0.045030,0.042133,0.035061,NaN,0.021291,0.051239,0.021023,0.038475,0.041751,0.032091
2003-02-28,NaN,0.030761,0.036602,0.042970,0.042106,0.033406,NaN,0.020968,0.049642,0.020183,0.037702,0.040185,0.030862
2003-03-31,NaN,0.030536,0.037642,0.039602,0.040662,0.035846,NaN,0.023387,0.050895,0.021027,0.037697,0.041202,0.032165
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-30,0.043538,0.016547,0.010547,0.021676,0.021544,0.022938,0.023062,0.006303,0.002752,0.005886,0.013793,-0.008969,0.010376
2022-12-31,0.045032,0.015152,0.009672,0.021593,0.020613,0.021749,0.024608,0.005594,0.001354,0.005306,0.012604,-0.010298,0.007769
2023-01-31,0.045688,0.014201,0.009618,0.021692,0.019527,0.019785,0.024872,0.005352,-0.000396,0.006128,0.012105,-0.009448,0.005583


# Momentum

In [ ]:
# Translate computeSortWeights into python
def compute_sort_weights(sort_variable, n_longs, n_shorts, long_high_values):
    """
    Generates portfolio weights based on sort_variable. The function ignores assets for which the sort variable is
    missing (NaN). All such assets get a weight of zero in the portfolio. n_longs and n_shorts denote the number of
    assets held long and short. When long_high_values is True, assets that have the highest values for sort_variable
    are held long and those with the lowest values are held short. Otherwise, the opposite holds.
    """
    # Find the assets with the highest and lowest values of the sort variable
    if long_high_values:
        list_of_longs = np.argpartition(-sort_variable, n_longs)[:n_longs]
        list_of_shorts = np.argpartition(sort_variable, n_shorts)[:n_shorts]
    else:
        list_of_longs = np.argpartition(sort_variable, n_longs)[:n_longs]
        list_of_shorts = np.argpartition(-sort_variable, n_shorts)[:n_shorts]

    # Assign the weights to assets in the list of longs and shorts
    n_assets = len(sort_variable)
    weights = np.zeros(n_assets)
    weights[list_of_longs] = 1 / n_longs
    weights[list_of_shorts] = -1 / n_shorts

    return weights

In [ ]:
def compute_sort_weights_GPT_2(sort_variable, n_longs, n_shorts, long_high_values):
    """
    Generates portfolio weights based on sort_variable. The function ignores assets for which the sort variable is
    missing (NaN). All such assets get a weight of zero in the portfolio. n_longs and n_shorts denote the number of
    assets held long and short. When long_high_values is True, assets that have the highest values for sort_variable
    are held long and those with the lowest values are held short. Otherwise, the opposite holds.
    """
    # Find the assets with the highest and lowest values of the sort variable
    if long_high_values:
        sort_variable_sorted = -sort_variable  # For long high values, sort descending
    else:
        sort_variable_sorted = sort_variable   # For long low values, sort ascending
    non_nan_assets = ~np.isnan(sort_variable_sorted)
    non_nan_count = np.count_nonzero(non_nan_assets)
    
    # Check if n_shorts is zero
    if n_shorts == 0:
        if non_nan_count < n_longs:
            n_longs = non_nan_count
        list_of_longs = np.argpartition(sort_variable_sorted, -n_longs)[-n_longs:]
        weights = np.zeros(len(sort_variable))
        weights[list_of_longs] = 1 / n_longs
        return weights
    
    # If n_shorts is not zero, handle normally
    if non_nan_count < n_longs:
        n_longs = non_nan_count
        n_shorts = 0
    elif n_shorts > 0 and non_nan_count < n_longs + n_shorts:
        n_shorts = non_nan_count - n_longs
    
    if long_high_values:
        list_of_longs = np.argpartition(sort_variable_sorted, -n_longs)[-n_longs:]
        list_of_shorts = np.argpartition(sort_variable_sorted, n_shorts)[:n_shorts]
    else:
        list_of_longs = np.argpartition(sort_variable_sorted, n_longs)[:n_longs]
        list_of_shorts = np.argpartition(sort_variable_sorted, -n_shorts)[-n_shorts:]

    # Assign the weights to assets in the list of longs and shorts
    n_assets = len(sort_variable)
    weights = np.zeros(n_assets)
    weights[list_of_longs] = 1 / n_longs
    weights[list_of_shorts] = -1 / n_shorts

    return weights

In [ ]:
def compute_sort_weights_GPT(sort_variable, n_longs, n_shorts, long_high_values):
    """
    Generates portfolio weights based on sort_variable. The function ignores assets for which the sort variable is
    missing (NaN). All such assets get a weight of zero in the portfolio. n_longs and n_shorts denote the number of
    assets held long and short. When long_high_values is True, assets that have the highest values for sort_variable
    are held long and those with the lowest values are held short. Otherwise, the opposite holds.
    """
    # Find the assets with the highest and lowest values of the sort variable
    if long_high_values:
        sort_variable_sorted = -sort_variable  # For long high values, sort descending
    else:
        sort_variable_sorted = sort_variable   # For long low values, sort ascending
    non_nan_assets = ~np.isnan(sort_variable_sorted)
    non_nan_count = np.count_nonzero(non_nan_assets)
    if non_nan_count < n_longs:
        n_longs = non_nan_count
        n_shorts = 0
    elif n_shorts > 0 and non_nan_count < n_longs + n_shorts:
        n_shorts = non_nan_count - n_longs
    
    if long_high_values:
        list_of_longs = np.argpartition(sort_variable_sorted, -n_longs)[-n_longs:]
        if n_shorts > 0:
            list_of_shorts = np.argpartition(sort_variable_sorted, n_shorts)[:n_shorts]
        else:
            list_of_shorts = []
    else:
        list_of_longs = np.argpartition(sort_variable_sorted, n_longs)[:n_longs]
        if n_shorts > 0:
            list_of_shorts = np.argpartition(sort_variable_sorted, -n_shorts)[-n_shorts:]
        else:
            list_of_shorts = []

    # Assign the weights to assets in the list of longs and shorts
    n_assets = len(sort_variable)
    weights = np.zeros(n_assets)
    weights[list_of_longs] = 1 / n_longs
    if n_shorts > 0:
        weights[list_of_shorts] = -1 / n_shorts

    return weights

In [ ]:
# first we take the average of the three Swiss indices



# s1 = ETF_price["Switzerland 1"]
# s2 = ETF_price["Switzerland 2"]
# s3 = ETF_price["Switzerland 3"]

#ETF_price.drop(["Switzerland 2", "Switzerland 3"], axis = 1, inplace = True)
#ETF_price.loc["2003-11-28":,"Switzerland 1"] = (np.nanmean([s1.loc["2003-11-28":], s2.loc["2003-11-28":], s3.loc["2003-11-28":]], axis = 0))
#ETF_price.rename(columns={"Switzerland 1": "Switzerland"}, inplace = True)
#ETF_price

In [215]:
# Create Monthly Returns out of ETF Prices
ETF_price = ETF.copy()
ETF_returns = ETF_price.pct_change()
ETF_ret = ETF_returns.tail(-1)
ETF_ret

,Germany,France,Spain,Italy,UK,Swiss comb,China,Australia,Japan,USA,Canada,Singapore,India
Datum,,,,,,,,,,,,,
2000-01-31,-0.004171,-0.005380,-0.003788,-0.002261,-0.008643,NaN,NaN,-0.002667,0.001062,NaN,NaN,NaN,NaN
2000-02-29,0.006663,0.005979,0.005704,0.005947,0.017969,NaN,NaN,0.021341,-0.005362,NaN,NaN,0.010291,NaN
2000-03-31,0.016454,0.016604,0.016070,0.013889,0.011709,NaN,NaN,0.012913,0.003724,NaN,NaN,0.004587,NaN
2000-04-28,-0.000372,-0.000371,-0.000186,0.000278,0.010394,NaN,NaN,0.003988,0.003371,NaN,NaN,0.005115,NaN
2000-05-31,0.004095,0.002878,0.002698,0.000185,0.000872,NaN,NaN,0.012299,0.006855,NaN,NaN,-0.005588,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-30,0.020856,0.021247,0.020407,0.027423,0.028590,-0.003167,NaN,0.014057,-0.007105,0.026820,0.028205,0.024462,0.035617
2022-12-30,-0.040014,-0.046631,-0.039934,-0.045044,-0.040348,-0.026188,NaN,-0.022093,-0.017501,-0.007295,-0.016807,0.013594,-0.014761
2023-01-31,0.016672,0.022153,0.018742,0.032246,0.026417,0.014935,NaN,0.029014,-0.004038,0.026849,0.029945,-0.001737,0.016199


In [398]:
# Create Weights for Momentum

# Prep Work
nAssets = len(ETF_price.columns) # 13 Countries
nMonths = len(ETF_price) # 280 Months of prices (31.12.1999 - 31.03.2023)
lookbackStart = 12
lookbackEnd = 1
firstMonth = lookbackStart + 1 # we can only start computing weights in 13.month bc we have 12 months lookback 
nLongs = 5
nShorts = 5
momLongWeights = pd.DataFrame(np.zeros((nMonths, nAssets)))
momLSWeights = pd.DataFrame(np.zeros((nMonths, nAssets)))
long_weight = np.divide(1,nLongs)
short_weight = np.divide(1,nShorts)*(-1)

In [261]:
# return the n columns with maximum value
def n_max_row(df, n):
    return df.apply(lambda x: x.nlargest(n).index.tolist(), axis=1)

def n_min_row(df, n):
    return df.apply(lambda x: x.nsmallest(n).index.tolist(), axis=1)

In [399]:
 # Loop to get the Long weights and the Long/Short Weights

for month in range(firstMonth, nMonths):
     pastReturns = pd.DataFrame(np.divide(ETF_price.iloc[month - lookbackEnd, :], ETF_price.iloc[month - lookbackStart, :]) - 1)
     highReturns = pastReturns.nlargest(nLongs,columns=pastReturns.columns)
     lowReturns = pastReturns.nsmallest(nShorts,columns=pastReturns.columns)
     low_index = lowReturns.index
     high_index = highReturns.index
     pRet_index = pastReturns.index

     high_index_numbers = pRet_index.get_indexer(high_index)
     low_index_numbers = pRet_index.get_indexer(low_index)
     # Long Weights only
     momLongWeights.iloc[month,high_index_numbers] = long_weight
     # Long/Short Weights only
     momLSWeights.iloc[month,high_index_numbers] = long_weight
     momLSWeights.iloc[month,low_index_numbers] = short_weight

# Set Index and Column Names again
momLongWeights = momLongWeights.set_index(ETF_price.index).copy()
momLongWeights.columns = ETF_price.columns

momLSWeights = momLSWeights.set_index(ETF_price.index).copy()
momLSWeights.columns = ETF_price.columns


In [395]:
momLongWeights

In [396]:
momLongWeights


,Germany,France,Spain,Italy,UK,Swiss comb,China,Australia,Japan,USA,Canada,Singapore,India
Datum,,,,,,,,,,,,,
1999-12-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2000-01-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2000-02-29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2000-03-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2000-04-28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-30,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.2,0.2,0.0,0.0,0.2,0.2
2022-12-30,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.2,0.2,0.0,0.0,0.2,0.2
2023-01-31,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.2,0.0,0.2,0.2,0.2


In [390]:
ETF_price

,Germany,France,Spain,Italy,UK,Swiss comb,China,Australia,Japan,USA,Canada,Singapore,India
Datum,,,,,,,,,,,,,
1999-12-31,62.067645,64.435090,72.015994,70.007121,5503.796927,NaN,NaN,21.837369,1880.306992,NaN,NaN,NaN,NaN
2000-01-31,61.808760,64.088403,71.743181,69.848868,5456.227342,NaN,NaN,21.779136,1882.304759,NaN,NaN,77.006985,NaN
2000-02-29,62.220622,64.471584,72.152401,70.264282,5554.270588,NaN,NaN,22.243926,1872.210934,NaN,NaN,77.799433,NaN
2000-03-31,63.244394,65.542055,73.311859,71.240175,5619.305533,NaN,NaN,22.531163,1879.183388,NaN,NaN,78.156314,NaN
2000-04-28,63.220859,65.517726,73.298219,71.259956,5677.714276,NaN,NaN,22.621007,1885.517717,NaN,NaN,78.556093,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-30,127.120400,139.419800,175.723700,174.678700,12820.950700,119.675300,NaN,65.660100,2571.300000,26.8264,25.7149,141.970000,33.3650
2022-12-30,122.033800,132.918500,168.706300,166.810400,12303.645900,116.541233,NaN,64.209500,2526.300000,26.6307,25.2827,143.900000,32.8725
2023-01-31,124.068400,135.863100,171.868200,172.189400,12628.667900,118.281800,NaN,66.072500,2516.100000,27.3457,26.0398,143.650000,33.4050


In [329]:
high_index_numbers

array([11,  7,  8])

In [331]:
low_index

Index(['UK', 'France', 'Germany'], dtype='object')

In [332]:
low_index_numbers

array([4, 1, 0])

In [288]:
pastReturns

,0
Germany,-0.139789
France,-0.147838
Spain,-0.133440
Italy,-0.122494
UK,-0.186922
Swiss comb,-0.046518
China,NaN
Australia,-0.035041
Japan,-0.043433
USA,-0.070847


In [126]:
# Data Cleanup -> Returns in sync with weights!
momLongWeights_cut = momLongWeights.iloc[firstMonth:, :] # cut out first 13 months -> first row now is weight calculated at 31.01.2000; keep extra row at end tor Turnover
momLSWeights_cut = momLSWeights.iloc[firstMonth:, :] # cut out first 13 months -> first row now is weight calculated at 31.01.2000; keep extra row at end tor Turnover
ETF_ret_cut = ETF_ret.iloc[firstMonth :, :] # Not firstMonth+1 bc we dropped one row above in ret calculation -> we need Returns from 28.02.2000
dates4Fig = ETF_ret_cut.index
nMonths_actual = nMonths - (firstMonth+1)


In [128]:
momLongWeights_cut.iloc[ : -1, :]

,Germany,France,Spain,Italy,UK,Swiss comb,China,Australia,Japan,USA,Canada,Singapore,India
Datum,,,,,,,,,,,,,
2001-01-31,0.0,0.0,0.0,0.0,0.0,0.2,0.2,0.0,0.0,0.2,0.2,0.0,0.2
2001-02-28,0.0,0.0,0.0,0.0,0.0,0.2,0.2,0.0,0.0,0.2,0.2,0.0,0.2
2001-03-30,0.0,0.0,0.0,0.0,0.0,0.2,0.2,0.0,0.0,0.2,0.2,0.0,0.2
2001-04-30,0.0,0.0,0.0,0.0,0.0,0.2,0.2,0.0,0.0,0.2,0.2,0.0,0.2
2001-05-31,0.0,0.0,0.0,0.0,0.0,0.2,0.2,0.0,0.0,0.2,0.2,0.0,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-31,0.0,0.0,0.0,0.0,0.0,0.2,0.2,0.2,0.2,0.0,0.0,0.0,0.2
2022-11-30,0.0,0.0,0.0,0.0,0.0,0.2,0.2,0.0,0.2,0.0,0.0,0.2,0.2
2022-12-30,0.0,0.0,0.0,0.0,0.0,0.2,0.2,0.0,0.2,0.0,0.0,0.2,0.2


In [129]:
ETF_ret_cut

,Germany,France,Spain,Italy,UK,Swiss comb,China,Australia,Japan,USA,Canada,Singapore,India
Datum,,,,,,,,,,,,,
2001-02-28,0.005184,0.004814,0.004832,0.004641,0.002488,NaN,NaN,0.006978,0.009370,NaN,NaN,0.010654,NaN
2001-03-30,0.007779,0.007928,0.007782,0.007670,0.000407,NaN,NaN,0.005505,0.005019,NaN,NaN,-0.002504,NaN
2001-04-30,-0.013703,-0.013483,-0.012580,-0.010553,-0.015788,NaN,NaN,-0.011707,0.000122,NaN,NaN,0.004076,NaN
2001-05-31,0.001319,0.001752,0.003163,0.003409,-0.003189,NaN,NaN,-0.004580,0.006702,NaN,NaN,0.006033,NaN
2001-06-29,0.008606,0.008571,0.007445,0.007580,-0.000622,NaN,NaN,0.000884,0.002494,NaN,NaN,0.002787,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-30,0.020856,0.021247,0.020407,0.027423,0.028590,-0.003167,NaN,0.014057,-0.007105,0.026820,0.028205,0.024462,0.035617
2022-12-30,-0.040014,-0.046631,-0.039934,-0.045044,-0.040348,-0.026188,NaN,-0.022093,-0.017501,-0.007295,-0.016807,0.013594,-0.014761
2023-01-31,0.016672,0.022153,0.018742,0.032246,0.026417,0.014935,NaN,0.029014,-0.004038,0.026849,0.029945,-0.001737,0.016199


In [ ]:
nMonths_actual

In [ ]:
# Calculate strategy Returns no TC
stratReturnsNoTC = pd.DataFrame(np.zeros((nMonths_actual, 2)))
# note we only take weights rows until end -1 bc we left one to much in
stratReturnsNoTC.iloc[:, 1] = np.sum(np.multiply(ETF_ret_cut, momLongWeights_cut.iloc[ : -1, :]))
stratReturnsNoTC.iloc[:, 2] = np.sum(np.multiply(ETF_ret_cut, momLongWeights_cut.iloc[ : -1, :])) # RF monthly must be added!!!




In [ ]:
# we start off by calculating the returns
ETF_returns = ETF_price.pct_change()

# here we convert the values to numeric in order to take the maximum afterwards
ETF_returns = ETF_returns.apply(pd.to_numeric)

# here we aggregate the past 11 months returns
past_11 = (ETF_returns+1).rolling(11).apply(np.prod)-1 # accumulate returns over 11 months 
past_11

# Carry

In [109]:
# for carry we will now calculate the differences between the long-term yields (2,5,10,20,30) and short-term yields (1M, 3M)
# we will create a dataframe for the 1M and the 3M

Carry_3m = pd.DataFrame(columns = Yield_long.columns, index = Yield_long.index)
avg_Carry_3m = pd.DataFrame(0, columns = Yield_short.columns, index = Yield_short.index)

runner = 0
counter = 0
for i in range(0,len(Yield_long.columns)):
    if counter == 5:
        runner += 1
        counter = 0
    Carry_3m.iloc[:,i] = Yield_long.iloc[:,i] - Yield_short.iloc[:,runner]
    counter += 1
    
for i in range(0,len(Yield_short.columns)):
    avg_Carry_3m.iloc[:,i] = np.nanmean([Carry_3m.iloc[:,i*5], Carry_3m.iloc[:,i*5+1], Carry_3m.iloc[:,i*5+2], Carry_3m.iloc[:,i*5+3], Carry_3m.iloc[:,i*5+4]], axis = 0)

avg_Carry_3m = avg_Carry_3m / 100

<ipython-input-109-b8644c83feae>:13: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  Carry_3m.iloc[:,i] = Yield_long.iloc[:,i] - Yield_short.iloc[:,runner]
<ipython-input-109-b8644c83feae>:17: RuntimeWarning: Mean of empty slice
  avg_Carry_3m.iloc[:,i] = np.nanmean([Carry_3m.iloc[:,i*5], Carry_3m.iloc[:,i*5+1], Carry_3m.iloc[:,i*5+2], Carry_3m.iloc[:,i*5+3], Carry_3m.iloc[:,i*5+4]], axis = 0)


# Sort function to create the buckets

In [219]:
def pmp_Sort(sort_variable, n_longs, n_shorts, long_high_values=1, equal_weights_ls=False):
    """
    Generates portfolio weights based on sort_variable. The function ignores assets for which the sort variable is
    missing (NaN). All such assets get a weight of zero in the portfolio. n_longs and n_shorts denote the number of
    assets held long and short. When long_high_values is True, assets that have the highest values for sort_variable
    are held long and those with the lowest values are held short. Otherwise, the opposite holds.
    """
    # Find the assets with the highest and lowest values of the sort variable
    if long_high_values:
        sort_variable_sorted = sort_variable  # For long high values, sort descending
    else:
        sort_variable_sorted = -sort_variable   # For long low values, sort ascending
    non_nan_assets = ~np.isnan(sort_variable_sorted)
    non_nan_count = np.count_nonzero(non_nan_assets)
    
    # Check if n_shorts is zero
    if n_shorts == 0:
        if non_nan_count < n_longs:
            n_longs = non_nan_count
        list_of_longs = np.argpartition(sort_variable_sorted, -n_longs)[-n_longs:]
        weights = np.zeros(len(sort_variable))
        weights[list_of_longs] = 1 / n_longs
        return weights

    # If n_shorts is not zero, handle normally
    if non_nan_count < n_longs:
        n_longs = non_nan_count
        n_shorts = 0
    elif n_shorts > 0 and non_nan_count < n_longs + n_shorts:
        n_shorts = non_nan_count - n_longs
        
    if equal_weights_ls == True:
        n_shorts = non_nan_count // 2
        n_longs = non_nan_count // 2
        
    

    tmp_long = list(sort_variable_sorted)
    tmp_long = [-10000000 if pd.isna(x) else x for x in tmp_long]
    list_of_longs = np.argsort(tmp_long)[-5:]
        
    tmp_short = list(sort_variable_sorted)
    tmp_short = [100000000 if pd.isna(x) else x for x in tmp_short]
    list_of_shorts = np.argsort(tmp_short)[:5]
    

    # Assign the weights to assets in the list of longs and shorts
    n_assets = len(sort_variable)
    weights = np.zeros(n_assets)
    weights[list_of_longs] = 1 / n_longs
    weights[list_of_shorts] = -1 / n_shorts

    return weights


In [120]:
# Value
Value_weights = pd.DataFrame(columns = Real_yield_avg.columns, index = Real_yield_avg.index)

for i,k in enumerate(Real_yield_avg.index):
    temp = pmp_Sort(Real_yield_avg.iloc[i,:], 5,5)
    Value_weights.iloc[i,:] = temp


# Momentum





# Carry
Carry_weights = pd.DataFrame(columns = avg_Carry_3m.columns, index = avg_Carry_3m.index)

for i,k in enumerate(avg_Carry_3m.index):
    temp = pmp_Sort(avg_Carry_3m.iloc[i,:], 5, 5)
    Carry_weights.iloc[i,:] = temp

Carry_weights

# Benchmarking

In [ ]:
benchmark = df[["MSCI_EU"]]
benchmark = benchmark.loc["1999-12-31 00:00:00":"2022-09-30 00:00:00"]
benchmark

In [ ]:
bm_mtl_ret = mtl_ret = benchmark.pct_change().resample("M").agg(lambda x: (1+x).prod()-1) # pct_change creates ordinary returns, resample Monthly and aggregating with the (1+x) -1 formula to get monthly ordinary returns
bm_mtl_ret


In [ ]:
#compare momentum result vs benchmark
comparison = pd.DataFrame(bm_mtl_ret)
comparison["mom_ret"] = profits
comparison.rename(columns={ comparison.columns[0]: "bench_ret" }, inplace = True)
comparison["bench_indexed"] = comparison["bench_ret"].add(1).cumprod()
comparison["mom_indexed"] = comparison["mom_ret"].add(1).cumprod()
comparison


In [ ]:
comparison

# Comparison Plot

In [ ]:
# Plotting
fig, ax = plt.subplots()
ax.plot(comparison.index,comparison.mom_indexed, label="Momentum", color="red")
ax.plot(comparison.index,comparison.bench_indexed, label="MSCI EU", color="blue")
ax.legend(loc="best")
ax.set_ylabel('Monthly Returns')
ax.set_title("Momentum long/short 5 sectors monthly rebalancing vs MSCI EU, indexed 31.12.1999")